In [104]:
# Imports
import pandas as pd
import firebase_admin
from firebase_admin import credentials, db
import bcrypt

In [105]:
# Connect to FireBase DB
cred = credentials.Certificate('credentials.json')

if not firebase_admin._apps:
    cred = credentials.Certificate('credentials.json')
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://nature-notebook-db-default-rtdb.firebaseio.com/'
    })

In [106]:
# User class to store local data
class User:
    def __init__(self, user_id: int, username: str, email: str):
        self.user_id = user_id
        self.username = username
        self.email = email
        self.settings =  self.init_setting_map() # Dict for settings
        self.permissions = set()  # Set of granted permissions

    def init_setting_map(self):
        dict = {}
        dict["light_mode"] = True
        dict["user_active"] = True
        # TO DO: add more settings
        return dict

    def update_setting(self, key: str, value):
        self.settings[key] = value

    def get_setting(self, key: str):
        return self.settings.get(key, None)

    def add_permission(self, permission: str):
        self.permissions.add(permission)

    def remove_permission(self, permission: str):
        self.permissions.discard(permission)

    def check_permission(self, permission: str) -> bool:
        return permission in self.permissions

    def __repr__(self):
        return (f"User(id={self.user_id}, username='{self.username}')")

In [107]:
# Password hashing and checking functions
def hash_pass_text(password_text):
    # First time hashing password using bcrypt; the salt is saved into the hash itself; slow hash to prevent malicious brute force attacks
    password_bytes = password_text.encode('utf-8')
    return bcrypt.hashpw(password_bytes, bcrypt.gensalt(14))

def check_password(password_check, password_true):
    # Check hashed password
    password_check = password_check.encode('utf-8')
    password_true = password_true.encode('utf-8')
    return bcrypt.checkpw(password_check, password_true)


In [108]:
# Returns total user database
def get_users():
    ref = db.reference('users/')
    snapshot = ref.get()

    data = []    
    for username, user_info in snapshot.items():
        user_info['username'] = username  # Add username to the dictionary
        data.append(user_info)

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)

    columns = ['username'] + [col for col in df.columns if col != 'username'] # Rearrange columns to get username in front
    df = df[columns]

    return df # Return data

In [109]:
# Get all users from DB example
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,0000000000000000000000000000000000000000000000...
1,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,1001101011001001101011001001101011001001101011...
2,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,0000000000000000000000000000000000000000000000...
3,stevenpi12,stevenp8@illinois.edu,True,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,0000000000000000000000000000000000000000000000...


In [ ]:
# Bit string operations for species found
def init_bitstring():
    return '0' * 200

def set_bit(bitstring, i, value):    
    lst = list(bitstring)
    lst[i - 1] = '1' if value else '0'
    return ''.join(lst)

def get_bit(bitstring: str, i: int) -> bool:    
    return bitstring[i - 1] == '1'

def get_all_true_indices(bitstring: str) -> list:
    return [(i + 1) for i, c in enumerate(bitstring) if c == '1']


In [ ]:
# Functions to add and remove user
def add_user(username, email, password, species_found = init_bitstring(), password_is_hashed = False):
    ref = db.reference(f"users/{username}")  
    if ref.get() is not None: # Check if user already exists
        return 
    
    # Create user dict to push
    user = {
        'email': email,
        'password_hash' : hash_pass_text(password).decode('utf-8') if not password_is_hashed else password,
        'is_active': True,
        'species_found' : species_found
    }
    
    ref.set(user)

def remove_user(username):
    ref = db.reference(f"users/{username}")  
    ref.delete()

In [112]:
# Add and remove user example
add_user('jethalal', 'jijaji@sundar.com', 'JalebiMaster420')
add_user('chungus', 'bingus@yahoo.co.in', 'EvilLarry')
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,0000000000000000000000000000000000000000000000...
1,chungus,bingus@yahoo.co.in,True,$2b$14$gbWJ2Yg4SHnW9UJ19mZIU.UKYlOc9IhfF46guQt...,0000000000000000000000000000000000000000000000...
2,jethalal,jijaji@sundar.com,True,$2b$14$fE7fibYvqIcMMKLY3Fd5r.tevprrZgQNy00aGsr...,0000000000000000000000000000000000000000000000...
3,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,1001101011001001101011001001101011001001101011...
4,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,0000000000000000000000000000000000000000000000...
5,stevenpi12,stevenp8@illinois.edu,True,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,0000000000000000000000000000000000000000000000...


In [113]:
remove_user('jethalal')
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,0000000000000000000000000000000000000000000000...
1,chungus,bingus@yahoo.co.in,True,$2b$14$gbWJ2Yg4SHnW9UJ19mZIU.UKYlOc9IhfF46guQt...,0000000000000000000000000000000000000000000000...
2,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,1001101011001001101011001001101011001001101011...
3,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,0000000000000000000000000000000000000000000000...
4,stevenpi12,stevenp8@illinois.edu,True,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,0000000000000000000000000000000000000000000000...


In [114]:
# General getter function
def get_info(username, what):
    path_to_node = f'users/{username}/{what}'
    ref = db.reference(path_to_node)
    snapshot = ref.get()
    if snapshot is not None:    # If info exists, return it
        return snapshot

# Specific getters
def get_email(username):
    return get_info(username, 'email')

def get_species_found(username):
    return get_info(username, 'species_found')

def get_species_found_list(username):
    return get_all_true_indices(get_species_found(username))

def get_password(username):
    return get_info(username, 'password_hash')

def get_active_status(username):
    return get_info(username, 'is_active')


In [115]:
# Getters example
get_email('carrot_and_pea_cookie')

'nkalode2@illinois.edu'

In [116]:
# Function to validate password of an user 
def validate_password(username, pass_check):
    password_hash = get_password(username)
    return check_password(pass_check, password_hash)

In [117]:
# Validate password example
print(validate_password('chungus', 'EvilLarry'))
print(validate_password('chungus', 'EvilBarry'))

True
False


In [ ]:
# Update observation helper function
def update_info(username, new_what, what): 
    ref = db.reference(f'users/{username}')  
    updates = { what : new_what } 
    ref.update(updates) # Push update

# Update observation's data functions
def update_username(username, new_username):
    add_user(new_username, get_email(username), get_password(username), get_species_found(username), True)
    remove_user(username)

def update_email(username, new_email):
    update_info(username, new_email, 'email')

def update_password(username, new_pass):
    new_hash = hash_pass_text(new_pass).decode('utf-8')
    update_info(username, new_hash, 'password_hash')

def update_active(username):
    curr = get_active_status(username)
    update_info(username, not curr, 'is_active')

# Add a species id newly found by the user
def change_species_found_status(username, species_id, status):
    curr_found = get_species_found(username)
    bit = '1' if status == True else '0'
    if (get_bit(curr_found, species_id) != bit):
        curr_found = set_bit(curr_found, species_id, status)
        update_info(username, curr_found, 'species_found')

def add_species_found(username, species_id):
    change_species_found_status(username, species_id, True)

def remove_species_found(username, species_id):
    change_species_found_status(username, species_id, False)
        
# Reset the species found by the user
def remove_all_species_found(username):
    update_info(username, init_bitstring(), 'species_found')

In [119]:
# Update email, active example & remove species found example
update_email('chungus', 'billa@gmail.com')
update_active('stevenpi12')
remove_all_species_found('lagumists047')
get_users()

,username,email,is_active,password_hash,species_found
0,carrot_and_pea_cookie,nkalode2@illinois.edu,True,$2b$14$ygN1P1ijoa6KmATx37rx4.a0hbdbL3mebK21yzm...,0000000000000000000000000000000000000000000000...
1,chungus,billa@gmail.com,True,$2b$14$gbWJ2Yg4SHnW9UJ19mZIU.UKYlOc9IhfF46guQt...,0000000000000000000000000000000000000000000000...
2,lagumists047,ks129@illinois.edu,True,$2b$14$xlr0N4XxXFEd2pzvmYnoreUfObcgC3jcr3W.bdW...,0000000000000000000000000000000000000000000000...
3,linger1109,ishaang5@illinois.edu,True,$2b$14$P0nPrPSn74tC3VFfqpJ6teJ6ZBGN7KiFKQlbEIN...,0000000000000000000000000000000000000000000000...
4,stevenpi12,stevenp8@illinois.edu,False,$2b$14$0k/kWhjxsvDutmPi5iB/FeCghJOEibCbteMFA7c...,0000000000000000000000000000000000000000000000...


In [120]:
# Kartikey found some birds (Add species example)
remove_user('chungus')
for i in range(1, 48, 3):
    add_species_found('lagumists047', i)

get_species_found_list('lagumists047')

[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46]

In [121]:
# User forget about a species
remove_species_found('lagumists047', 10)
get_species_found_list('lagumists047')

[1, 4, 7, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46]

In [122]:
# Function to add birds to DB
def add_bird(id, name=None, description=None):
    ref = db.reference(f"birds/{id}")  
    if ref.get() is not None: # Check if bird already exists
        return 
    
    # Create bird dict to push
    birdie = {
        'name': name,
        'description' : description
    }
    
    ref.set(birdie)

# Get info of the birds from DB
def get_bird_info(id):
    ref = db.reference(f"birds/{id}")  
    if ref.get() is None: # Check if bird doesn't exist
        return 
    snapshot = ref.get()
    data = []  
    for _, bird_info in snapshot.items():
        data.append(bird_info)
    data.reverse()
    return data

In [123]:
# Get bird info example
print(get_bird_info(1))

['Black footed Albatross', 'The black-footed albatross (Phoebastria nigripes) is a large seabird of the albatross family Diomedeidae from the North Pacific. All but 2.5% of the population is found among the Northwestern Hawaiian Islands. It is one of three species of albatross that range in the northern hemisphere, nesting on isolated tropical islands. Unlike many albatrosses, it is dark plumaged. Black-footed albatrosses are a type of albatross that belong to family Diomedeidae of the order Procellariiformes, along with shearwaters, fulmars, storm petrels, and diving petrels. They share certain identifying features. First, they have nasal passages that attach to the upper bill called naricorns. Although the nostrils on the albatross are on the sides of the bill. The bills of Procellariiformes are also unique in that they are split into between seven and nine horny plates. Finally, they produce a stomach oil made up of wax esters and triglycerides that is stored in the proventriculus. 

In [124]:
# Deployed all 200 birds on FireBase; don't need to run again

# df = pd.read_csv('birds.csv')

# import re
# def extract_int(str):
#     match = re.search(r'\d+', str)
#     return int(match.group()) if match else None

# for index, row in df.iterrows():
#     add_bird(extract_int(row['ID']), row[' Name'], row[' Description'])